In [ ]:
from sklearn import datasets
from scipy.io.arff import loadarff
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd


In [ ]:
forest = datasets.fetch_covtype()

In [ ]:
type(forest)

In [ ]:
X, X_test, y, y_test = train_test_split(
    forest.data, forest.target == 2, test_size=0.3, random_state=1234)

In [ ]:
print(X.shape, y.shape, X_test.shape, y_test.shape)

## Maintenant un peu de xgboost!

### D'abord un tout petit

In [ ]:
import numpy as np
def logit(p):
    return np.log(p) - np.log(1 - p)

def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    # 'tree_method': 'gpu_hist'
}

num_round = 4
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


In [ ]:
X_test[0, [0,25,35,15]]

In [ ]:
leaves = bst.predict(dtest, pred_leaf=True)
z = bst.predict(dtest)
leaves[:3], z[:3], inv_logit(0.388039+ 0.354413)

In [ ]:
leaves[400], y[400], X_test[400, [0,7,3]]

In [ ]:
bst.dump_model("bst_2_trees")
! cat bst_2_trees

In [ ]:
X.shape, y.shape, X_test.shape, y_test.shape

In [ ]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


In [ ]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


In [ ]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


In [ ]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


In [ ]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 40
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

In [ ]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'device': 'gpu'
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5,
                verbose_eval=200)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))